In [1]:
import matplotlib.pyplot as plt
from pprint import pprint
import sys
sys.path.insert(0, '..')
from fashion import utils
from fashion import preprocessing
import numpy as np

# show all results of the notebook not just the last one
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
# get (my private) API key
api_key = utils.get_api_key()

# load the shops data
shops = preprocessing.load_shops('../data/20200120_filiali.csv')
shops.head()

#oop

\Users\odhra\Documents\GitHub\OSG\fashion\data\Unique_product_counts.pkl
Loading Unique product counts in stores
\Users\odhra\Documents\GitHub\OSG\fashion\data\Total_product_counts.pkl
Loading Total product counts in stores


,StoreKey,Franchise,StoreType,ZipCode,City,NUniqueProductsSold,NTotalProductsSold
0,2d459c389e1a0561a2953b95cfdc0caa,True,Store,95024,ACIREALE,30008,137059
1,c93364d5937332fea742adf1bb0fd67e,False,Store,37010,AFFI,31718,140788
2,8316b325ea6fd3bdeb794ffb649eb556,False,Store,80021,AFRAGOLA,34949,202128
3,c359def3032833a2ebbc12c8948a18f3,False,Store,51031,AGLIANA,9395,42147
4,293e777d87dea6fe28930c91bb2f99fa,True,Store,86081,AGNONE,11930,34045


In [3]:
sales = preprocessing.load_sales('../data/20200120_sales17.csv')
sales.head()

,StoreKey,ReceiptKey,Date,Hour,EAN,Volume,NetIncome
0,2d459c389e1a0561a2953b95cfdc0caa,160217,20161210,1318,2088988001033,1,19.95
1,2d459c389e1a0561a2953b95cfdc0caa,160253,20161210,1634,2088988001033,1,19.95
2,2d459c389e1a0561a2953b95cfdc0caa,160319,20161210,1911,2088988001057,1,19.95
3,2d459c389e1a0561a2953b95cfdc0caa,160330,20161210,1931,2088988001064,1,19.95
4,2d459c389e1a0561a2953b95cfdc0caa,160499,20161211,1747,2088986001042,1,19.95


In [3]:
items = preprocessing.load_products('../data/20200120_barcode.csv')
items.head()

,EAN,ProductID,ColorCode,ColorDescription,Size,Gender,ItemCode,ItemDescription,ProductCategory,Season,...,SizeID,OriginalListedPrice,StandardCostOfGoods,ReceivedItems,IntroductionPeriod,Lifecycle,PriceRange,FashionContent,ItemWeight,InnovationContent
0,2.000010e+12,9,UN,other,PZ,UN,BS,BUONO SCONTO,UN,AI,...,1,5.0,0.01,0,,,0,NC,NC,NC
1,2.000000e+12,1,UN,other,PZ,UN,GF,GIFT CARD,UN,AI,...,1,15.0,5.00,-7,,,0,NC,NC,NC
2,2.000000e+12,2,UN,other,PZ,UN,GF,GIFT CARD,UN,AI,...,1,25.0,8.33,-13,,,0,NC,NC,NC
3,2.000000e+12,2,UN,other,PZ,UN,GF,GIFT CARD,UN,AI,...,1,0.0,0.00,0,,,0,NC,NC,NC
4,2.000000e+12,3,UN,other,PZ,UN,GF,GIFT CARD,UN,AI,...,1,50.0,16.66,-1,,,0,NC,NC,NC


In [4]:
item_cluster = items.drop(['EAN', 'ColorCode'],1)

item_cluster.head()

,ProductID,ColorDescription,Size,Gender,ItemCode,ItemDescription,ProductCategory,Season,Year,SizeID,OriginalListedPrice,StandardCostOfGoods,ReceivedItems,IntroductionPeriod,Lifecycle,PriceRange,FashionContent,ItemWeight,InnovationContent
0,9,other,PZ,UN,BS,BUONO SCONTO,UN,AI,1990,1,5.0,0.01,0,,,0,NC,NC,NC
1,1,other,PZ,UN,GF,GIFT CARD,UN,AI,1990,1,15.0,5.00,-7,,,0,NC,NC,NC
2,2,other,PZ,UN,GF,GIFT CARD,UN,AI,1990,1,25.0,8.33,-13,,,0,NC,NC,NC
3,2,other,PZ,UN,GF,GIFT CARD,UN,AI,1990,1,0.0,0.00,0,,,0,NC,NC,NC
4,3,other,PZ,UN,GF,GIFT CARD,UN,AI,1990,1,50.0,16.66,-1,,,0,NC,NC,NC


In [5]:
item_cluster = item_cluster.replace(np.nan, 'NaN', regex=True)

In [10]:
#just try to assert that all str should be int. Look for a pandas function for this!

i = 0
for x in item_cluster['PriceRange']:
    if type(x) == str:
#         print(x)
        i += 1

print(i)

448818


In [11]:
#so not available gender isn't properly coded here - check if this corresponds to other NaNs that you removed! the like 40k of them. 

i = 0
for x in item_cluster['Gender']:
    if type(x) == float:
        
        i += 1

print(i)

19933


In [6]:
categorical_variables = [] 
category_names = []
for i,x in enumerate(item_cluster.columns):
    type_set = set()
    print(i,x, len(set(item_cluster[x])), type(list(item_cluster[x])[0]))
    for y in item_cluster[x]:
        type_set.add(type(y))
    print(type_set)
    if type(list(item_cluster[x])[0]) == str:
        categorical_variables.append(i)
        category_names.append(x)
    if type(list(item_cluster[x])[0]) == int:
        categorical_variables.append(i)
        category_names.append(x)

0 ProductID 42798 <class 'int'>
{<class 'int'>}
1 ColorDescription 51 <class 'str'>
{<class 'str'>}
2 Size 82 <class 'str'>
{<class 'str'>}
3 Gender 8 <class 'str'>
{<class 'str'>}
4 ItemCode 130 <class 'str'>
{<class 'str'>}
5 ItemDescription 129 <class 'str'>
{<class 'str'>}
6 ProductCategory 8 <class 'str'>
{<class 'str'>}
7 Season 3 <class 'str'>
{<class 'str'>}
8 Year 9 <class 'int'>
{<class 'int'>}
9 SizeID 15 <class 'int'>
{<class 'int'>}
10 OriginalListedPrice 142 <class 'float'>
{<class 'float'>}
11 StandardCostOfGoods 1460 <class 'float'>
{<class 'float'>}
12 ReceivedItems 7213 <class 'int'>
{<class 'int'>}
13 IntroductionPeriod 9 <class 'str'>
{<class 'str'>}
14 Lifecycle 8 <class 'str'>
{<class 'str'>}
15 PriceRange 123 <class 'str'>
{<class 'str'>}
16 FashionContent 4 <class 'str'>
{<class 'str'>}
17 ItemWeight 4 <class 'str'>
{<class 'str'>}
18 InnovationContent 5 <class 'str'>
{<class 'str'>}


In [20]:
print(category_names)

['Product_identifier', 'Color_description', 'Size_code', 'Gender', 'Item', 'Item_description', 'Product_category_code', 'season_code', 'year', 'colour_ID', 'size_ID', 'Recieved_items', 'Introduction_period', 'rotation', 'price_range', 'price_range', 'fashion_content', 'seasonality', 'innovation_content']


In [8]:
categorical_frame = item_cluster.filter(items=category_names, axis=1)

In [9]:
print(categorical_frame.head())

   ProductID ColorDescription Size Gender ItemCode ItemDescription  \
0          9            other   PZ     UN       BS    BUONO SCONTO   
1          1            other   PZ     UN       GF       GIFT CARD   
2          2            other   PZ     UN       GF       GIFT CARD   
3          2            other   PZ     UN       GF       GIFT CARD   
4          3            other   PZ     UN       GF       GIFT CARD   

  ProductCategory Season  Year  SizeID  ReceivedItems IntroductionPeriod  \
0              UN     AI  1990       1              0                      
1              UN     AI  1990       1             -7                      
2              UN     AI  1990       1            -13                      
3              UN     AI  1990       1              0                      
4              UN     AI  1990       1             -1                      

  Lifecycle PriceRange FashionContent ItemWeight InnovationContent  
0                    0             NC         NC     

In [10]:
continuous_categories = list(set(item_clusterter.columns) - set(category_names))

In [11]:
print(continuous_categories)

['OriginalListedPrice', 'StandardCostOfGoods']


In [12]:
continuous_frame = item_cluster.filter(items=continuous_categories, axis=1)

In [15]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=20, random_state=0, verbose=2)

continuous_clusters = kmeans.fit_predict(continuous_frame)

Initialization complete
start iteration
done sorting
end inner loop
Iteration 0, inertia 277875.18431249197
start iteration
done sorting
end inner loop
Iteration 1, inertia 259330.55241778967
start iteration
done sorting
end inner loop
Iteration 2, inertia 257094.0222335776
start iteration
done sorting
end inner loop
Iteration 3, inertia 255774.44373581308
start iteration
done sorting
end inner loop
Iteration 4, inertia 254970.26072190813
start iteration
done sorting
end inner loop
Iteration 5, inertia 254473.89741148913
start iteration
done sorting
end inner loop
Iteration 6, inertia 254352.11216731046
center shift 1.195311e-03 within tolerance 4.803245e-03
Initialization complete
start iteration
done sorting
end inner loop
Iteration 0, inertia 283833.2882907151
start iteration
done sorting
end inner loop
Iteration 1, inertia 268165.6823406662
start iteration
done sorting
end inner loop
Iteration 2, inertia 265058.2708081524
start iteration
done sorting
end inner loop
Iteration 3, ine

In [17]:
print(kmeans.cluster_centers_)

[[12.9494348   3.69064724]
 [59.95       16.06157604]
 [19.95592035  6.41659664]
 [ 2.45308751  0.93679333]
 [ 6.15943138  1.76551869]
 [29.98125964  9.29670027]
 [91.75380859 22.7128125 ]
 [40.03208595 10.75561564]
 [10.11930566  2.9649187 ]
 [25.01537266  6.93322762]
 [17.24486693  4.94975983]
 [49.87269915 13.16748647]
 [ 8.03968367  2.27658395]
 [79.94251135 19.86838124]
 [69.95       18.1032844 ]
 [15.05169443  4.29762552]
 [35.18265169  9.50173986]
 [ 4.53028167  1.31962399]
 [29.99167875  7.19117126]
 [19.93854309  5.22963323]]


In [26]:
from kmodes.kmodes import KModes

km = KModes(n_clusters=20, init='Huang', n_init=100, verbose=2)

clusters = km.fit_predict(categorical_frame)

print(km.cluster_centroids_)


Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 1, iteration: 1/100, moves: 117221, cost: 3802623.0
Run 1, iteration: 2/100, moves: 7110, cost: 3802613.0
Run 1, iteration: 3/100, moves: 18, cost: 3802613.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 2, iteration: 1/100, moves: 114781, cost: 3748335.0
Run 2, iteration: 2/100, moves: 27281, cost: 3748332.0
Run 2, iteration: 3/100, moves: 1, cost: 3748332.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 3, iteration: 1/100, moves: 120444, cost: 3811229.0
Run 3, iteration: 2/100, moves: 36833, cost: 3811229.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 4, iteration: 1/100, moves: 104709, cost: 3807788.0
Run 4, iteration: 2/100, moves: 19707, cost: 3796901.0
Run 4, iteration: 3/100, moves: 9868, cost: 3796894.0
Run 4, iteration: 4/100, moves: 38, cost: 3796894.0
Init: initializing centroids

Starting iterations...
Run 33, iteration: 1/100, moves: 126904, cost: 3738148.0
Run 33, iteration: 2/100, moves: 16088, cost: 3738148.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 34, iteration: 1/100, moves: 102251, cost: 3804491.0
Run 34, iteration: 2/100, moves: 5972, cost: 3804491.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 35, iteration: 1/100, moves: 121201, cost: 3749273.0
Run 35, iteration: 2/100, moves: 29999, cost: 3749209.0
Run 35, iteration: 3/100, moves: 11, cost: 3749209.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 36, iteration: 1/100, moves: 129596, cost: 3765112.0
Run 36, iteration: 2/100, moves: 31792, cost: 3755159.0
Run 36, iteration: 3/100, moves: 5311, cost: 3755128.0
Run 36, iteration: 4/100, moves: 14, cost: 3755128.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 37, iteration: 1/100, moves: 135409, cost

Init: initializing clusters
Starting iterations...
Run 65, iteration: 1/100, moves: 139139, cost: 3799649.0
Run 65, iteration: 2/100, moves: 28981, cost: 3795776.0
Run 65, iteration: 3/100, moves: 2655, cost: 3795776.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 66, iteration: 1/100, moves: 124589, cost: 3820475.0
Run 66, iteration: 2/100, moves: 13894, cost: 3812081.0
Run 66, iteration: 3/100, moves: 10196, cost: 3808876.0
Run 66, iteration: 4/100, moves: 536, cost: 3808876.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 67, iteration: 1/100, moves: 92098, cost: 3942695.0
Run 67, iteration: 2/100, moves: 34490, cost: 3928288.0
Run 67, iteration: 3/100, moves: 16146, cost: 3921964.0
Run 67, iteration: 4/100, moves: 1609, cost: 3921964.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 68, iteration: 1/100, moves: 126982, cost: 3766079.0
Run 68, iteration: 2/100, moves: 18443

Run 95, iteration: 3/100, moves: 5679, cost: 3803386.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 96, iteration: 1/100, moves: 168715, cost: 3739813.0
Run 96, iteration: 2/100, moves: 34351, cost: 3739813.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 97, iteration: 1/100, moves: 113623, cost: 3719009.0
Run 97, iteration: 2/100, moves: 22800, cost: 3717834.0
Run 97, iteration: 3/100, moves: 625, cost: 3717834.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 98, iteration: 1/100, moves: 100579, cost: 3828283.0
Run 98, iteration: 2/100, moves: 24226, cost: 3826422.0
Run 98, iteration: 3/100, moves: 1380, cost: 3826422.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 99, iteration: 1/100, moves: 141449, cost: 3787605.0
Run 99, iteration: 2/100, moves: 17663, cost: 3787584.0
Run 99, iteration: 3/100, moves: 9, cost: 3787584.0
Init: initia

In [45]:
print(len(clusters),len(items.EAN))

categorical_export_array = items.EAN.to_frame()


448818 448818


In [46]:
categorical_export_array['clusters'] = clusters

In [51]:
print(categorical_export_array.head)

export = categorical_export_array.to_csv(r'C:\Users\odhra\Documents\GitHub\OSG\fashion\data\categorical_clusters.csv')

<bound method NDFrame.head of                  EAN  clusters
0       2.000010e+12         2
1       2.000000e+12         2
2       2.000000e+12         2
3       2.000000e+12         2
4       2.000000e+12         2
...              ...       ...
448813  2.026120e+12         3
448814  2.026120e+12         3
448815  2.026120e+12         3
448816  2.026120e+12         3
448817  2.026120e+12         3

[448818 rows x 2 columns]>


In [40]:
print(type(categorical_export_array))

<class 'pandas.core.frame.DataFrame'>


In [ ]:

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 14, 15, 16, 16, 17, 18, 19]

In [23]:
from kmodes.kprototypes import KPrototypes 

K=10

KP = KPrototypes(n_clusters=2, init='Cao', verbose=1)

KP.fit(cluster_dropna, categorical = categorical_variables)

# KP.fit_predict(cluster_dropna, categorical = categorical_variables)


# oh hey look another bug. This time it's with 

Init: initializing centroids
Init: initializing clusters


IndexError: index 1 is out of bounds for axis 1 with size 1

AttributeError: '{}' object has no attribute 'cluster_centroids_' because the model is not yet fitted.

In [4]:
import pandas as pd
import numpy as np
import time
import csv
import matplotlib.pyplot as plt
import seaborn as sns
import random


from sklearn import cluster
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.model_selection import cross_val_score, train_test_split, GridSearchCV
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE, MDS, Isomap
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier

import warnings
warnings.filterwarnings("ignore")



In [4]:
from kmodes.kmodes import KModes

In [ ]:
#Build store cluster model - these can be your categories. Categories for what?

#you could try to identify outliers in these clusters - is there a store that is underperforming in a cluster? if so, 
#in what ways does it differ in terms of allocation of goods etc?



#https://pypi.org/project/pgeocode/ - can convert italy ('it') zip to lat & long







In [ ]:
#Build item cluster model that is relatively well-cleaned. 

#https://towardsdatascience.com/clustering-on-mixed-type-data-8bbd0a2569c3



# If your data contains both numeric and categorical variables, the best way to carry out clustering on the dataset is to create principal components of the dataset and use the principal component scores as input into the clustering.

# Remember that u can always get principal components for categorical variables using a multiple correspondence analysis (MCA), which will give principal components, and you can get then do a separate PCA for the numerical variables, and use the combined as input into your clustering.

# OR u could use the R package called FactorMineR or PCAmix to carry Factor analysis of mixed data, with the output being principal components, and then using the principal components as input into your clustering.

# Remember that clustering and principal components are doing almost similar thing in a simplistic way they're both distances, e.g Euclidean distance.

#https://datascience.stackexchange.com/questions/22/k-means-clustering-for-mixed-numeric-and-categorical-data - K-modes approach to clustering.

# you could also use a K-modes approach to clustering, which might have some value - https://medium.com/@davidmasse8/unsupervised-learning-for-categorical-data-dd7e497033ae